### Step 1: Install necessary packages
 ```pip install transformers datasets torch scikit-learn pandas tqdm ```

### Step 2: Prepare the Training Data
Ensure the dataset (```assembly_conduit_wire_train.csv```) has a "Description" column and a corresponding "Label" column (manually labeled)

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load and preprocess dataset
df = pd.read_csv('./assets/classified_invoices.csv')

# Ensure there is manually labeld data
df = df[['Description', 'Label']] # Columns: 'Description' (text), 'Label' (target) 

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(df['Description'].tolist(), df['Label'].tolist())


### Step 3: Encode Labels
Convert text labels into numeric labels

In [8]:
from sklearn.preprocessing import LabelEncoder

# Convert class labels to numbers
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)

# Save label mapping for later use
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(f'Label Mapping: {label_mapping}')


Label Mapping: {np.str_('Conduit'): np.int64(0), np.str_('Exclude'): np.int64(1), np.str_('Wire'): np.int64(2)}


### Step 4: Tokenize the data
Use BERT's tokenizer to process the text descriptions

In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize descriptions
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)



### Step 5: Create a dataset for PyTorch
BERT requires data in PyTorch tensor format

In [11]:
import torch

class ConduitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    

# Convert encodings into datasets
train_dataset = ConduitDataset(train_encodings, train_labels)
val_dataset = ConduitDataset(val_encodings, val_labels)


### Step 6: Load Pretrained BERT Model
We'll fine-tune BERT for classification

In [12]:
from transformers import BertForSequenceClassification

num_labels = len(label_mapping) # Number of unique categories

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Step 7: Train the BERT Model
Train the model using PyTorch's Trainer API

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import TrainerCallback


class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=2):
        self.patience = patience
        self.best_loss = float("inf")
        self.counter = 0
    
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        val_loss = metrics["eval_loss"]
        print(f"Epoch {state.epoch:.2f} Validation Loss: {val_loss:.6f}")
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            print(f"Early Stopping COUnter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                print("🚨 Early stopping triggered! Training will stop.")
                control.should_training_stop = True
    

training_args = TrainingArguments(
    output_dir="./bert_conduit_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(patience=2)]
)

trainer.train()
print(f"Best model loaded from: {trainer.state.best_model_checkpoint}")


x:\OwenEC\Projects\InvoiceAssembly\.venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.479500,0.374264
2,0.212900,0.233189
3,0.160200,0.162310
4,0.049700,0.180749
5,0.064900,0.138873
6,0.048000,0.131471
7,0.001000,0.217939


Epoch 1.00 Validation Loss: 0.374264
Epoch 2.00 Validation Loss: 0.233189
Epoch 3.00 Validation Loss: 0.162310
Epoch 4.00 Validation Loss: 0.180749
Early Stopping COUnter: 1/2
Epoch 5.00 Validation Loss: 0.138873
Epoch 6.00 Validation Loss: 0.131471
Epoch 7.00 Validation Loss: 0.217939
Early Stopping COUnter: 1/2


: 